In [49]:
import queries_km_servicio_diario as qr
import psycopg2
import pandas as pd
import numpy as np  
import xlsxwriter
from datetime import datetime, timedelta,date
import copy
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")
pd.set_option('display.max_columns', None)
inicio_d = datetime.today().date() - timedelta(days = 1)
fin_d    = datetime.today().date()# - timedelta(days = 1)
#%run "validacion_km_diario_oficial_reproceso_3.ipynb"
import nbformat
with open("C:\\Users\\Tomas Retamal\\OneDrive - BUSES HUALPEN\\Documentos\\Proyectos\\TomasRetamal\\Opti\\KM Vacío\\KM Pablo\\validacion_km_diario_oficial_reproceso_3.ipynb", encoding="utf8") as f:
    nb2 = nbformat.read(f, as_version=4)

for cell2 in nb2.cells:
    if cell2.cell_type == "code":
        exec(cell2.source)

2023-02-16
Conexión exitosa


<string>:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


vhc_corregida_can
SE CONECTO BIEN


<string>:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


vhc_corregida_gps


In [50]:
fecha_inicial = inicio_d.strftime("%Y-%m-%d")
fecha_fin = fin_d.strftime("%Y-%m-%d")

# Definición de directorios

In [51]:
#dir = r"C:\Users\Richard Salgado\Desktop\Kilometraje\Kilometraje_total\Kilometraje_consolidado\Kilometraje_oficial_septiembre"
#dir = r'C:\Users\Tomas Retamal\OneDrive - BUSES HUALPEN\Documentos\Proyectos\TomasRetamal\Opti\KM Vacío\KM Pablo'
dir = r'C:\Users\Tomas Retamal\OneDrive - BUSES HUALPEN\Documentos\Proyectos\PabloJerez\respaldo\Kilometraje\Kilometraje_total\Kilometraje_consolidado\Kilometraje_oficial_septiembre\validaciones_individuales_reproceso_2'
# output
# dir = dir + r'\validaciones_individuales_reproceso_2'
dir_out_val_can = rf'{dir}\validacion_can_{fecha_inicial}_{fecha_fin}.xlsx'
dir_out_val_gps = rf'{dir}\validacion_gps_{fecha_inicial}_{fecha_fin}.xlsx'
dir_out_val_comb = rf'{dir}\validacion_comb_{fecha_inicial}_{fecha_fin}.xlsx'
dir_out_val_tall = rf'{dir}\validacion_taller_{fecha_inicial}_{fecha_fin}.xlsx'
dir_out_val_chl = rf'{dir}\validacion_checklist_{fecha_inicial}_{fecha_fin}.xlsx'

In [52]:
# Parte heredada de Pablo
cc = 'Unnamed: 0'
reportes_1=pd.read_excel(rf'{dir}\validacion_gps_{fecha_inicial}_{fecha_fin}.xlsx','Comparacion')
del reportes_1[cc]
validacion_gps=reportes_1.values.tolist()

reportes_3=pd.read_excel(rf'{dir}\validacion_can_{fecha_inicial}_{fecha_fin}.xlsx','Comparacion')
del reportes_3[cc]
reportes_3['numero_interno'] = reportes_3['numero_interno'].apply(lambda x: x.strip().split(' ')[0] if type(x)==str else x)
validacion_can=reportes_3.values.tolist()

# Funciones viajes

In [53]:
def etiquetar_errores(x):
    velocidad = 150
    if float(x.viaje_distancia_gps) < 0 or float(x.viaje_distancia_can) < 0:
        x.etiqueta = 'Negativo'
    elif float(x.viaje_distancia_gps) > 2000 or float(x.viaje_distancia_can) > 2000:
        x.etiqueta = 'Excesivo'
    elif ((x.viaje_fecha_termino_real - x.viaje_fecha_inicio_real).total_seconds() / 3600) * velocidad < x.viaje_distancia_can or ((x.viaje_fecha_termino_real - x.viaje_fecha_inicio_real).total_seconds()/3600)*velocidad < x.viaje_distancia_gps:
        x.etiqueta = 'Excesivo'
    
    else:
        if pd.isna(x.viaje_km_inicial_gps) == True or pd.isna(x.viaje_km_final_gps) == True:
            
            if pd.isna(x.viaje_km_inicial_can)==True or pd.isna(x.viaje_km_final_can) == True:
                x.etiqueta = 'Faltante gps y faltante can'
            else:
                x.etiqueta = 'Faltante gps'

        else:
            if pd.isna(x.viaje_km_inicial_can)==True or pd.isna(x.viaje_km_final_can) == True:
                x.etiqueta = 'Existe gps y faltante can'
            else:
                x.etiqueta = 'Existe gps y existe can'
    return x

## Obtener viajes y sus observaciones

In [54]:
## Función hecha por Pablo
def obtener_viajes_con_obs(fecha_ini,fecha_fin):
    conn_sql = psycopg2.connect(user = "tretamal",
                            password = "LrtA0XYloG",
                            host = "192.168.40.239",
                            port = "5432",
                            database = "bidb",
                            options="-c search_path=dbo,taller")

    # Obtención de viajes desde bidb
    df = pd.read_sql(qr.query_viajes_bidb(str(inicio_d), str(fin_d)), conn_sql)
    df["viaje_id"] = df["viaje_id"].astype("int64")
    df['etiqueta'] = None
    df = df.apply(lambda x: etiquetar_errores(x), axis = 1)
    conn_sql = psycopg2.connect(user = "cons_paul",
                                password = "DoNPau2746l",
                                host = "db-hualpen-com-reader.cm5wbxahj6z3.us-east-2.rds.amazonaws.com",
                                port = "5432",
                                database = "hualpen_ser2_prod")
    # Obtención de viajes con observaciones desde cursor
    df_3= pd.read_sql(qr.query_observaciones(str(inicio_d), str(fin_d)), conn_sql)

    # merge entre viajes bidb y viajes con observaciones
    viajes_con_obs = pd.merge(df,df_3,on = 'viaje_id', how = 'left')

    # las observaciones traidas son las correspondientes a cambios de móvil
    # nos interesa saber cuándo ocurrió el cambio
    viajes_con_obs["momento_cambio_movil"] = pd.Series(dtype='str')
    viajes_con_obs["momento_cambio_movil"] = viajes_con_obs.apply(lambda x: "Antes del viaje" if (x.fecha_creacion < x.viaje_fecha_inicio_real) else "Durante el viaje" if (x.fecha_creacion > x.viaje_fecha_inicio_real) else None, axis = 1)
    viajes_con_obs = viajes_con_obs.sort_values(["viaje_id", "fecha_creacion"], ignore_index=True)

    # puede haber más de una observación por viaje, lo que genera registros duplicados
    # los eliminamos quedándonos con el registro de la observación más reciente
    viajes_con_obs.drop_duplicates(subset = ["viaje_id"], keep = "last", inplace = True)
    viajes_con_obs = viajes_con_obs.reset_index(drop=True)
    
    return viajes_con_obs


## Obtener viajes de dos días

En esta función, obtenemos solo los viajes que terminan al día siguiente (desde los viajes obtenidos en celda anterior)

In [55]:
def viajes_dos_dias(df):
    df["flag_planificado"] = df.apply(lambda x: 1 if ((x.viaje_fecha_inicio.date() != x.viaje_fecha_termino.date()) and ((x.viaje_fecha_termino.time().hour + x.viaje_fecha_termino.time().minute) != 0)) else 0, axis = 1)
    df["flag_real"]        = df.apply(lambda x: 1 if ( (pd.isna(x.viaje_fecha_inicio_real.date()) == False) and (pd.isna(x.viaje_fecha_termino_real.date()) == False) and ((x.viaje_fecha_termino_real.date() - x.viaje_fecha_inicio_real.date()).days == 1) and ((x.viaje_fecha_termino_real.time().hour + x.viaje_fecha_termino_real.time().minute) != 0)) else 0, axis = 1)
    df_dos_dias = df[(df["flag_real"] == 1)].reset_index(drop = True)
    return df_dos_dias

## Odometro medianoche (can y gps)

In [56]:
def odometro_medianoche_can(df,fecha_ini, fecha_fin):
    #df["id_numero_interno"] = df.apply(lambda x: str(x.id) + x.numero_interno, axis = 1)
    conn_sql = psycopg2.connect(user = "cons_paul",
                                    password = "DoNPau2746l",
                                    host = "db-hualpen-com-reader.cm5wbxahj6z3.us-east-2.rds.amazonaws.com",
                                    port = "5432",
                                    database = "hualpen_com_prod")
    lista_odometros_medianoche_can_fin = []
    fuente_odometro_medianoche_can = []
    hora_evento_can = []
    moviles = []
    ids = []
    for index, row in df_dos_dias.iterrows():
        #print(movil)
        aux = vhc_corregida_can[(vhc_corregida_can["numero_interno"] == row["movil_numero_interno"]) &  (vhc_corregida_can["fecha_inicio"] == pd.to_datetime(str(row["viaje_fecha_termino_real"].date()) + " 00:00:00", format = "%Y-%m-%d %H:%M:%S"))].sort_values("fecha_inicio").reset_index(drop = True)
        ## si en vhc no existe el dato, vamos a buscarlo a evento telemetría
        cursor = conn_sql.cursor()
        # si el movil no está en vhc (df vacío), nos saltamos el dato
        try:
            a = pd.isna(aux.iloc[0]["can_inicio"]) # si aquí hay un error, significa que el df aux es vacío, es decir, el dato que buscamos no está en vhc
            # si a es true, debo ingresar a query_eventos
            # si a es false, no ingreso a query eventos
            # si hay un error, a no tiene valor, por lo que hay que asignárselo en el except
            # en dicho caso, el valor que se le asigna a a es irrelevante, ya que el error
            # implica que aux es vacío, por lo que sí o sí se entrará a quey_eventos.
            # Arbitrariamente la asignaré a false 
        except:
            a = False
        if aux.empty or a:
            command_set_timezone = '''
            set time zone 'America/Santiago'
            '''    
            query_eventos = '''
            select v.nombre 
            ,et.fecha_evento
            ,(et.atributos::json-> 'senial'->>'odom_km')::decimal as odometro_can
            ,case
                when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),12,1) = '2' then to_timestamp(concat(et.fecha_evento::date + 1, ' 00:00:00'), 'YYYY-MM-DD HH24:MI:SS') - et.fecha_evento
                when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),12,1) = '0' then et.fecha_evento - to_timestamp(concat(et.fecha_evento::date    , ' 00:00:00'), 'YYYY-MM-DD HH24:MI:SS')
            --    when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),13,1) = '0' then to_timestamp(concat(et.fecha_evento::date, ' 21:00:00'), 'YYYY-MM-DD HH24:MI:SS') - et.fecha_evento
            --    when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),13,1) = '1' then et.fecha_evento - to_timestamp(concat(et.fecha_evento::date    , ' 21:00:00'), 'YYYY-MM-DD HH24:MI:SS')
            end diferencia_medianoche
            from evento_telemetria et 
            left join vehiculo v on et.vehiculo_id =v.id 
            where et.fecha_evento >= '{}' and et.fecha_evento < '{}'
            and v.nombre = '{}'
            order by diferencia_medianoche
            '''.format(str(row["viaje_fecha_inicio_real"].date()) + " 23:50:00", str(row["viaje_fecha_termino_real"].date()) + " 00:10:00", row["movil_numero_interno"])#format(str(row["fecha_inicio_real"].date()) + " 20:50:00", str(row["fecha_inicio_real"].date()) + " 21:10:00", row["numero_interno"])
            # .format(str(row["fecha_inicio_real"].date()) + " 23:50:00", str(row["fecha_termino_real"].date()) + " 00:10:00", row["numero_interno"])
            cursor.execute(command_set_timezone)
            df_eventos = pd.read_sql(query_eventos, conn_sql)
            cursor.close()
                # me quedo con todos los registros que sí tienen odometro can
            df_eventos_filtrado = df_eventos[pd.isna(df_eventos["odometro_can"]) == False]
            # si al aplicar el filtro el df queda vacío, no hay odómetro, por lo que guardamos None
            if df_eventos_filtrado.empty:
                lista_odometros_medianoche_can_fin.append(None)
                fuente_odometro_medianoche_can.append(None)
                hora_evento_can.append(None)
            else:
                lista_odometros_medianoche_can_fin.append(df_eventos_filtrado.iloc[0]["odometro_can"])
                fuente_odometro_medianoche_can.append("et")
                hora_evento_can.append(df_eventos_filtrado.iloc[0]["fecha_evento"])
        else:
            lista_odometros_medianoche_can_fin.append(aux.iloc[0]["can_inicio"])
            fuente_odometro_medianoche_can.append("vhc")
            hora_evento_can.append(pd.to_datetime(fecha_fin + " 00:00:00"))
        ids.append(row["viaje_id"])
        moviles.append(row["movil_numero_interno"])
        #ids.append(row["id"])
        #moviles.append(row["numero_interno"])
        
    #data_medianoche = {'nombre': moviles, 'odometros_medianoche_can_dia_inicio': lista_odometros_medianoche_can_inicio, 'odometros_medianoche_can_dia_termino': lista_odometros_medianoche_can_fin}
    data_medianoche_can = {'viaje_id':ids, 'movil_numero_interno': moviles, 'odometro_medianoche_can': lista_odometros_medianoche_can_fin, 'fuente_odometro_medianoche_can':fuente_odometro_medianoche_can, 'hora_evento_can_medianoche':hora_evento_can}
    df_medianoche_can = pd.DataFrame(data_medianoche_can)
    #df_medianoche["hora_evento_can_medianoche"] = df_medianoche["hora_evento_can_medianoche"].apply(lambda x: x.tz_localize(None) if pd.isna(x) == False else x)
    #df_medianoche["hora_evento_gps_medianoche"] = df_medianoche["hora_evento_gps_medianoche"].apply(lambda x: x.tz_localize(None) if pd.isna(x) == False else x)

    return df_medianoche_can # df_medianoche#, df_vhc#moviles_agendados_dos_dias, lista_odometros, lista_eventos

In [57]:
def odometro_medianoche_gps(df,fecha_ini, fecha_fin):
    #df["id_numero_interno"] = df.apply(lambda x: str(x.id) + x.numero_interno, axis = 1)
    conn_sql = psycopg2.connect(user = "cons_paul",
                                    password = "DoNPau2746l",
                                    host = "db-hualpen-com-reader.cm5wbxahj6z3.us-east-2.rds.amazonaws.com",
                                    port = "5432",
                                    database = "hualpen_com_prod")
    lista_odometros_medianoche_gps_fin = []
    fuente_odometro_medianoche_gps = []
    hora_evento_gps = []
    moviles = []
    ids = []
    for index, row in df_dos_dias.iterrows():
        #print(movil)
        aux = vhc_corregida_gps[(vhc_corregida_gps["numero_interno"] == row["movil_numero_interno"]) &  (vhc_corregida_gps["fecha_inicio"] == pd.to_datetime(str(row["viaje_fecha_termino_real"].date()) + " 00:00:00", format = "%Y-%m-%d %H:%M:%S"))].sort_values("fecha_inicio").reset_index(drop = True)
        ## si en vhc no existe el dato, vamos a buscarlo a evento telemetría
        cursor = conn_sql.cursor()
        # si el movil no está en vhc (df vacío), nos saltamos el dato
        try:
            a = pd.isna(aux.iloc[0]["odo_inicio_gps"]) # si aquí hay un error, significa que el df aux es vacío, es decir, el dato que buscamos no está en vhc
            # si a es true, debo ingresar a query_eventos
            # si a es false, no ingreso a query eventos
            # si hay un error, a no tiene valor, por lo que hay que asignárselo en el except
            # en dicho caso, el valor que se le asigna a a es irrelevante, ya que el error
            # implica que aux es vacío, por lo que sí o sí se entrará a quey_eventos.
            # Arbitrariamente la asignaré a false 
        except:
            a = False
        if aux.empty or a:
            command_set_timezone = '''
            set time zone 'America/Santiago'
            '''    
            query_eventos = '''
            select v.nombre 
            ,et.fecha_evento
            ,(et.atributos::json-> 'senial'->>'odom_km')::decimal as odometro_gps
            ,case
                when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),12,1) = '2' then to_timestamp(concat(et.fecha_evento::date + 1, ' 00:00:00'), 'YYYY-MM-DD HH24:MI:SS') - et.fecha_evento
                when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),12,1) = '0' then et.fecha_evento - to_timestamp(concat(et.fecha_evento::date    , ' 00:00:00'), 'YYYY-MM-DD HH24:MI:SS')
            --    when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),13,1) = '0' then to_timestamp(concat(et.fecha_evento::date, ' 21:00:00'), 'YYYY-MM-DD HH24:MI:SS') - et.fecha_evento
            --    when substring(to_char(et.fecha_evento, 'YYYY-MM-DD HH24:MI:SS'),13,1) = '1' then et.fecha_evento - to_timestamp(concat(et.fecha_evento::date    , ' 21:00:00'), 'YYYY-MM-DD HH24:MI:SS')
            end diferencia_medianoche
            from evento_telemetria et 
            left join vehiculo v on et.vehiculo_id =v.id 
            where et.fecha_evento >= '{}' and et.fecha_evento < '{}'
            and v.nombre = '{}'
            order by diferencia_medianoche
            '''.format(str(row["viaje_fecha_inicio_real"].date()) + " 23:50:00", str(row["viaje_fecha_termino_real"].date()) + " 00:10:00", row["movil_numero_interno"])#format(str(row["fecha_inicio_real"].date()) + " 20:50:00", str(row["fecha_inicio_real"].date()) + " 21:10:00", row["numero_interno"])
            # .format(str(row["fecha_inicio_real"].date()) + " 23:50:00", str(row["fecha_termino_real"].date()) + " 00:10:00", row["numero_interno"])
            cursor.execute(command_set_timezone)
            df_eventos = pd.read_sql(query_eventos, conn_sql)
            cursor.close()
                # me quedo con todos los registros que sí tienen odometro gps
            df_eventos_filtrado = df_eventos[pd.isna(df_eventos["odometro_gps"]) == False]
            # si al aplicar el filtro el df queda vacío, no hay odómetro, por lo que guardamos None
            if df_eventos_filtrado.empty:
                lista_odometros_medianoche_gps_fin.append(None)
                fuente_odometro_medianoche_gps.append(None)
                hora_evento_gps.append(None)
            else:
                lista_odometros_medianoche_gps_fin.append(df_eventos_filtrado.iloc[0]["odometro_gps"])
                fuente_odometro_medianoche_gps.append("et")
                hora_evento_gps.append(df_eventos_filtrado.iloc[0]["fecha_evento"])
        else:
            lista_odometros_medianoche_gps_fin.append(aux.iloc[0]["odo_inicio_gps"])
            fuente_odometro_medianoche_gps.append("vhc")
            hora_evento_gps.append(pd.to_datetime(fecha_fin + " 00:00:00"))
        ids.append(row["viaje_id"])
        moviles.append(row["movil_numero_interno"])
        #ids.append(row["id"])
        #moviles.append(row["numero_interno"])
        
    #data_medianoche = {'nombre': moviles, 'odometros_medianoche_gps_dia_inicio': lista_odometros_medianoche_odo_inicio_gps, 'odometros_medianoche_gps_dia_termino': lista_odometros_medianoche_gps_fin}
    data_medianoche_gps = {'viaje_id':ids, 'movil_numero_interno': moviles, 'odometro_medianoche_gps': lista_odometros_medianoche_gps_fin, 'fuente_odometro_medianoche_gps':fuente_odometro_medianoche_gps, 'hora_evento_gps_medianoche':hora_evento_gps}
    df_medianoche_gps = pd.DataFrame(data_medianoche_gps)
    return df_medianoche_gps
    #df_medianoche["hora_evento_gps_medianoche"] = df_medianoche["hora_evento_gps_medianoche"].apply(lambda x: x.tz_localize(None) if pd.isna(x) == False else x)
    #df_medianoche[" materi

## Unir df de viajes con df de odometros medianoche

In [58]:
def unir_dataframes(df_con_obs, df_odometro_medianoche):
    # df_odometro_medianoche = df_odometro_medianoche.drop(columns = df_odometro_medianoche.columns[1:-2])#, inplace = True)
    df_merged = pd.merge(df_con_obs, df_odometro_medianoche, "left", on = ["viaje_id", "movil_numero_interno"])
    return df_merged

## Separar los viajes que corresponden en 2

In [59]:
def generar_nuevos_registros(df):
    dff = copy.deepcopy(df)
    dff["tipo_corte"] = dff.apply(lambda x: "inicio" if x.flag_real == 1 else None, axis = 1)
    #dff["km_dia"] = dff["odometro_medianoche"] - dff["km_inicial_can"]
    for index, row in dff.iterrows():
        if row["flag_real"] == 1:
            dff.loc[len(dff)] = row.tolist()[:-7] + [row["odometro_medianoche_gps"], row["fuente_odometro_medianoche_gps"], row["hora_evento_gps_medianoche"], row["odometro_medianoche_can"], row["fuente_odometro_medianoche_can"], row["hora_evento_can_medianoche"],"fin"]
    return dff

## Etiqueta la fuente del dato de odometro medianoche disponible

In [60]:
def etiquetar_corte(x):
    if x.flag_real == 1 and pd.isna(x.odometro_medianoche_gps) == False and pd.isna(x.odometro_medianoche_can) == False:
        x.etiqueta_corte = "Existe gps y existe can"
    elif x.flag_real == 1 and pd.isna(x.odometro_medianoche_gps) == False and pd.isna(x.odometro_medianoche_can) == True:
        x.etiqueta_corte = "Existe gps y faltante can"
    elif x.flag_real == 1 and pd.isna(x.odometro_medianoche_gps) == True and pd.isna(x.odometro_medianoche_can) == False:
        x.etiqueta_corte = "Faltante gps"
    elif x.flag_real == 1 and pd.isna(x.odometro_medianoche_gps) == True and pd.isna(x.odometro_medianoche_can) == True:
        x.etiqueta_corte = "Faltante gps y faltante can"
    else:
        x.etiqueta_corte = None
    return x

In [61]:
# Seleccionar km viajes en servicio
def seleccionar_km_servicio(x):
    if x.etiqueta_corte == None:
        if x.etiqueta == 'Existe gps y existe can':
            fuentes = {'CAN':float(x.viaje_distancia_can),'GPS':float(x.viaje_distancia_gps)}
            max_key = max(fuentes, key=lambda key: fuentes[key])
            x.km_seleccionado = max(float(x.viaje_distancia_can),float(x.viaje_distancia_gps))
            x.km_servicio_dia = x.km_seleccionado
            if max_key == 'CAN':
                x.fuente_km_seleccionado = "CAN"
            else:
                x.fuente_km_seleccionado = "GPS"

        elif x.etiqueta == 'Existe gps y faltante can':
            x.km_seleccionado = float(x.viaje_distancia_gps)
            x.km_servicio_dia = x.km_seleccionado
            x.fuente_km_seleccionado = "GPS"


        elif x.etiqueta == 'Faltante gps':
            x.km_seleccionado = float(x.viaje_distancia_can)
            x.km_servicio_dia = x.km_seleccionado
            x.fuente_km_seleccionado = "CAN"
        else:
            pass
            
            #return x

    elif x.etiqueta_corte != None:
        if x.etiqueta == 'Existe gps y existe can':
            if x.etiqueta_corte == 'Existe gps y existe can':
                fuentes = {'CAN':float(x.viaje_distancia_can),'GPS':float(x.viaje_distancia_gps)}
                max_key = max(fuentes, key=lambda key: fuentes[key])
                x.km_seleccionado = max(float(x.viaje_distancia_can),float(x.viaje_distancia_gps))
                if max_key == 'CAN':
                    if x.tipo_corte == 'inicio':
                        x.km_servicio_dia = x.odometro_medianoche_can - x.viaje_km_inicial_can
                    else:
                        x.km_servicio_dia = x.viaje_km_final_can - x.odometro_medianoche_can
                    x.fuente_km_seleccionado = "CAN"
                else:
                    if x.tipo_corte == 'inicio':
                        x.km_servicio_dia = x.odometro_medianoche_gps - x.viaje_km_inicial_gps
                    else:
                        x.km_servicio_dia = x.viaje_km_final_gps - x.odometro_medianoche_gps
                    x.fuente_km_seleccionado = "GPS"

            elif x.etiqueta_corte == 'Existe gps y faltante can':
                x.km_seleccionado = float(x.viaje_distancia_gps)
                if x.tipo_corte == 'inicio':
                    x.km_servicio_dia = x.odometro_medianoche_gps - x.viaje_km_inicial_gps
                else:
                    x.km_servicio_dia = x.viaje_km_final_gps - x.odometro_medianoche_gps
                x.fuente_km_seleccionado = "GPS"

            elif x.etiqueta_corte == 'Faltante gps':
                x.km_seleccionado = float(x.viaje_distancia_can)
                if x.tipo_corte == 'inicio':
                    x.km_servicio_dia = x.odometro_medianoche_can - x.viaje_km_inicial_can
                else:
                    x.km_servicio_dia = x.viaje_km_final_can - x.odometro_medianoche_can
                x.fuente_km_seleccionado = "CAN"
            else :
                x.km_seleccionado = max(float(x.viaje_distancia_can),float(x.viaje_distancia_gps))
                x.km_servicio_dia = None # más adelante, en este punto se repartirá el km de manera proporcional
                fuentes = {'CAN':float(x.viaje_distancia_can),'GPS':float(x.viaje_distancia_gps)}
                max_key = max(fuentes, key=lambda key: fuentes[key])
                if max_key == 'CAN':
                    x.fuente_km_seleccionado = "CAN"
                else:
                    x.fuente_km_seleccionado = "GPS"


        elif x.etiqueta == 'Existe gps y faltante can':
            if x.etiqueta_corte == 'Existe gps y faltante can' or x.etiqueta_corte == 'Existe gps y existe can':
                x.km_seleccionado = float(x.viaje_distancia_gps)
                if x.tipo_corte == 'inicio':
                    x.km_servicio_dia = x.odometro_medianoche_gps - x.viaje_km_inicial_gps
                else:
                    x.km_servicio_dia = x.viaje_km_final_gps - x.odometro_medianoche_gps
                x.fuente_km_seleccionado = "GPS"
            else:
                x.km_seleccionado = float(x.viaje_distancia_gps)
                x.km_servicio_dia = None
                x.fuente_km_seleccionado = "GPS"
                
        elif x.etiqueta == 'Faltante gps':
            if x.etiqueta_corte == 'Faltante gps' or x.etiqueta_corte == 'Existe gps y existe can':
                x.km_seleccionado = float(x.viaje_distancia_can)
                if x.tipo_corte == 'inicio':
                    x.km_servicio_dia = x.odometro_medianoche_can - x.viaje_km_inicial_can
                else:
                    x.km_servicio_dia = x.viaje_km_final_can - x.odometro_medianoche_can
                x.fuente_km_seleccionado = "CAN"
            else:
                x.km_seleccionado = float(x.viaje_distancia_can)
                x.km_servicio_dia = None
                x.fuente_km_seleccionado = "CAN"
        else:
            pass

    return x

In [62]:
def errores_inicio_y_fin_viajes(x):
    if x.flag_real == 1 and x.viaje_fecha_termino_real.date() < x.viaje_fecha_termino.date():
        x.error_odometro_medianoche = "Término real prematuro"
    elif x.flag_real == 1 and x.viaje_fecha_termino.date() < x.viaje_fecha_termino_real.date():
        x.error_odometro_medianoche = "Término real tardío"
    elif x.flag_real == 1 and x.viaje_fecha_inicio_real.date() < x.viaje_fecha_inicio.date():
        x.error_odometro_medianoche = "Inicio real prematuro"
    elif x.flag_real == 1 and x.viaje_fecha_inicio.date() < x.viaje_fecha_inicio_real.date():
        x.error_odometro_medianoche = "Inicio real tardío"
    return x

# ETL

In [63]:
df_con_obs = obtener_viajes_con_obs(str(inicio_d), str(fin_d))
df_dos_dias = viajes_dos_dias(df_con_obs)
# estas siguientes dos lineas no recuerdo porque las agregué, ahora dudo que sirvan de algo así que las dejaré comentadas
# df_dos_dias = copy.deepcopy(df_dos_dias[pd.isna(df_dos_dias["numero_interno"]) == False]) 
# df_dos_dias = copy.deepcopy(df_dos_dias[df_dos_dias["numero_interno"].str.startswith("BE") == False])
df_odometro_medianoche_can = odometro_medianoche_can(df_dos_dias, str(inicio_d), str(fin_d))
df_odometro_medianoche_gps = odometro_medianoche_gps(df_dos_dias, str(inicio_d), str(fin_d))
df_odometro_medianoche = pd.merge(df_odometro_medianoche_gps, df_odometro_medianoche_can, how = 'outer', on = ["viaje_id", "movil_numero_interno"])
df_unido = unir_dataframes(df_con_obs, df_odometro_medianoche)
df_nuevos_registros = generar_nuevos_registros(df_unido)
df_nuevos_registros["etiqueta_corte"] = None
df_nuevos_registros_ = df_nuevos_registros.apply(lambda x: etiquetar_corte(x), axis = 1)
df_nuevos_registros_["km_seleccionado"] = None
df_nuevos_registros_["km_servicio_dia"] = None
df_nuevos_registros_["fuente_km_seleccionado"] = None
df_nuevos_registros_n = df_nuevos_registros_.apply(lambda x: seleccionar_km_servicio(x), axis = 1)
df_nuevos_registros_n["fecha_km_servicio_dia"] = df_nuevos_registros_n.apply(lambda x: x.viaje_fecha_inicio_real.date() if x.tipo_corte == "inicio" else x.viaje_fecha_termino_real.date() if x.tipo_corte == "fin" else inicio_d, axis = 1)
df_nuevos_registros_n["id_registro"] = df_nuevos_registros_n.apply(lambda x: str(x.viaje_id) + "1" if x.tipo_corte == "inicio" else str(x.viaje_id) + "2" if x.tipo_corte == "fin" else str(x.viaje_id) + "0", axis = 1)
df_nuevos_registros_n["error_odometro_medianoche"] = None
df_nuevos_registros_n = df_nuevos_registros_n.apply(lambda x: errores_inicio_y_fin_viajes(x), axis = 1)
df_nuevos_registros_n["fecha_analisis"] = inicio_d

C:\Users\Tomas Retamal\AppData\Local\Temp\ipykernel_2396\3763490519.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(qr.query_viajes_bidb(str(inicio_d), str(fin_d)), conn_sql)
C:\Users\Tomas Retamal\AppData\Local\Temp\ipykernel_2396\3763490519.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3= pd.read_sql(qr.query_observaciones(str(inicio_d), str(fin_d)), conn_sql)
C:\Users\Tomas Retamal\AppData\Local\Temp\ipykernel_2396\737581636.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_eventos =

## DF final separado

In [64]:
df_nuevos_registros_n = df_nuevos_registros_n.reindex(columns = [
    "id_registro",
    "fecha_km_servicio_dia",
    "flag_planificado",
    "flag_real",
    "tipo_corte",
    "viaje_id",
    "movil_numero_interno",
    "sucursal_viaje",
    "descripcion_centro_costo",
    "servicio_historico_alias",
    "observacion",
    "viaje_fecha_inicio",
    "viaje_fecha_termino",
    "viaje_fecha_inicio_real",
    "viaje_fecha_termino_real",
    "viaje_estado",
    "etiqueta",
    "etiqueta_corte",
    "viaje_km_inicial_can",
    "viaje_km_final_can",
    "viaje_distancia_can",
    "odometro_medianoche_can",
    "fuente_odometro_medianoche_can",
    "hora_evento_can_medianoche",
    "viaje_km_inicial_gps",
    "viaje_km_final_gps",
    "viaje_distancia_gps",
    "odometro_medianoche_gps",
    "fuente_odometro_medianoche_gps",
    "hora_evento_gps_medianoche",
    "km_seleccionado",
    "km_servicio_dia",
    "error_odometro_medianoche",
    "fuente_km_seleccionado",
    "fecha_analisis",
    "fecha_creacion",
    "momento_cambio_movil",
    "viaje_confirmacion",
    "tipo_servicio",
    "viaje_fecha_inicio_gps",
    "viaje_fecha_termino_gps",
    "viaje_fecha_inicio_conductor",
    "viaje_fecha_termino_conductor",
    "viaje_fecha_inicio_administrador",
    "viaje_fecha_termino_adminiostrador"
])
df_nuevos_registros_n = df_nuevos_registros_n.rename(columns={'fecha_creacion':'fecha_creacion_observacion'})

In [65]:
df_nuevos_registros_n = df_nuevos_registros_n.replace({np.NaN:None})
df_nuevos_registros_n["flag_planificado"] = df_nuevos_registros_n["flag_planificado"].apply(lambda x: "True" if x == 1 else "False")
df_nuevos_registros_n["flag_real"] = df_nuevos_registros_n["flag_real"].apply(lambda x: "True" if x == 1 else "False")
df_nuevos_registros_n["viaje_confirmacion"] = df_nuevos_registros_n["viaje_confirmacion"].apply(lambda x: "True" if x == True else "False")
#df_nuevos_registros_n = copy.deepcopy(df_nuevos_registros_n[df_nuevos_registros_n["momento_cambio_movil"] != "Durante el viaje"])

## Crear tabla kilometrajes servicio diarios auxiliar

In [66]:
#try:    
#    conn_sql = psycopg2.connect(user = "paul_bello",
#                                password = "UAOZQ2Rr4t",
#                                host = "192.168.40.239",
#                                port = "5432",
#                                database = "bidb",
#                                options="-c search_path=dbo,taller")
#    
#    print('Conexión exitosa')
#except: 
#    print('Conexión fallida')
#cursor = conn_sql.cursor()
#query_crear = '''
#CREATE TABLE flota_pbello.bh_resumen_diario_km_servicio_auxiliar(
#    id INTEGER PRIMARY KEY,
#    )
#'''
#query_alter = '''
#    ALTER TABLE flota_pbello.bh_kilometrajes_servicio_dia_auxiliar
#    ADD COLUMN fecha_analisis DATE
#'''
##query_crear = ''' 
##    CREATE TABLE flota_pbello.bh_kilometrajes_servicio_dia_auxiliar (
##    id_registro INTEGER PRIMARY KEY,
##    fecha_km_servicio_dia DATE,
##    flag_planificado BOOLEAN,
##    flag_real BOOLEAN,
##    tipo_corte VARCHAR,
##    id INTEGER,
##    numero_interno VARCHAR,
##    sucursal VARCHAR,
##    ceco VARCHAR,
##    comercial VARCHAR,
##    alias VARCHAR,
##    observacion TEXT,
##    fecha_inicio TIMESTAMP,
##    fecha_termino TIMESTAMP,
##    fecha_inicio_real TIMESTAMP,
##    fecha_termino_real TIMESTAMP,
##    estado VARCHAR,
##    etiqueta VARCHAR,
##    etiqueta_corte VARCHAR,
##    km_inicial_can FLOAT,
##    km_final_can FLOAT,
##    distancia_can FLOAT,
##    odometro_medianoche_can FLOAT,
##    fuente_odometro_medianoche_can VARCHAR,
##    hora_evento_can_medianoche TIMESTAMP,
##    km_inicial_gps FLOAT,
##    km_final_gps FLOAT,
##    distancia_gps FLOAT,
##    odometro_medianoche_gps FLOAT,
##    fuente_odometro_medianoche_gps VARCHAR,
##    hora_evento_gps_medianoche TIMESTAMP,
##    km_seleccionado FLOAT,
##    km_servicio_dia FLOAT,
##    error_odometro_medianoche VARCHAR,
##    fuente_km_seleccionado VARCHAR
##);
###'''
#cursor.execute(query_alter)
#conn_sql.commit()
#cursor.close()
#conn_sql.close()

## Insertar datos en tabla creada en celda anterior

In [67]:
import psycopg2
import numpy
from psycopg2.extensions import register_adapter, AsIs
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)
def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)
register_adapter(numpy.float64, addapt_numpy_float64)
register_adapter(numpy.int64, addapt_numpy_int64)
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)


conn_sql = psycopg2.connect(user = "paul_bello",
                            password = "UAOZQ2Rr4t",
                            host = "192.168.40.239",
                            port = "5432",
                            database = "bidb",
                            options="-c search_path=dbo,taller")
cursor = conn_sql.cursor()
query_delete = f'''
    DELETE FROM flota_pbello.bh_kilometrajes_servicio_dia_auxiliar WHERE fecha_analisis = '{str(inicio_d)}';
    '''
query_alter = '''
    ALTER TABLE flota_pbello.bh_kilometrajes_servicio_dia_auxiliar
    DROP COLUMN ceco;
'''
cursor.execute(query_delete)#, tuple(list(df_nuevos_registros_n.iloc[i])))  
for i in range(len(df_nuevos_registros_n)):
    query_insert ='''
    INSERT INTO flota_pbello.bh_kilometrajes_servicio_dia_auxiliar (
    id_registro,
    fecha_km_servicio_dia,
    flag_planificado,
    flag_real,
    tipo_corte,
    id,
    numero_interno,
    sucursal,
    comercial,
    alias,
    observacion,
    fecha_inicio,
    fecha_termino,
    fecha_inicio_real,
    fecha_termino_real,
    estado,
    etiqueta,
    etiqueta_corte,
    km_inicial_can,
    km_final_can,
    distancia_can,
    odometro_medianoche_can,
    fuente_odometro_medianoche_can,
    hora_evento_can_medianoche,
    km_inicial_gps,
    km_final_gps,
    distancia_gps,
    odometro_medianoche_gps,
    fuente_odometro_medianoche_gps,
    hora_evento_gps_medianoche,
    km_seleccionado,
    km_servicio_dia,
    error_odometro_medianoche,
    fuente_km_seleccionado,
    fecha_analisis,
    fecha_creacion_observacion,
    momento_cambio_movil,
    confirmacion,
    tipo_servicio,
    viaje_fecha_inicio_gps,
    viaje_fecha_termino_gps,
    viaje_fecha_inicio_conductor,
    viaje_fecha_termino_conductor,
    viaje_fecha_inicio_administrador,
    viaje_fecha_termino_administrador)
    VALUES (
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s)
    '''
    cursor.execute(query_insert, tuple(list(df_nuevos_registros_n.iloc[i])))
    if i % 100 == 0:
        conn_sql.commit()      
#cursor.execute(query_alter,  tuple(list(df_nuevos_registros_n.iloc[i])))
conn_sql.commit()
cursor.close()
conn_sql.close()